# Module 1: Thao tác dữ liệu tín hiệu sinh học

## Bài 2: Lấy mẫu và dán nhãn cho dữ liệu

### BS. Lê Ngọc Khả Nhi

# Khởi hành

Thân chào các bạn đồng nghiệp, đây là bài thực hành thứ 2 trong Module 1 về Thao tác trên dữ liệu tín hiệu sinh học (Biosignal data wrangling).

Trong bài trước, chúng ta đã khởi động bước đầu tiên trong một thí nghiệm machine learning giả định, mà nguyên liệu là tín hiệu thô xuất ra từ thiết bị đa ký giấc ngủ. Ta đã biết về những cấu trúc dữ liệu chuỗi trong bài toán, và dùng một Python package để thi hành công đoạn tải dữ liệu này.

Ở bài này, Nhi sẽ giới thiệu tiếp về công đoạn lấy mẫu (sampling) và dán nhãn (labelling) để chuẩn bị dữ liệu đầu vào cho thí nghiệm. Để thực hiện công đoạn này, Nhi mở rộng package module1 với class mới là Data_Sampler().

Ta tiếp tục làm việc trên cùng gói dữ liệu như bài 1, có thể tải về từ link sau:
https://drive.google.com/file/d/1Md59XrQizIKCJ5G8_mwxbLNaBRN5nMm4/view?usp=sharing

Như đã giải thích trong bài trước, Nhi không trình bày code trong notebook, nhưng chuẩn bị trước tất cả function và class cần thiết trong một package 'module1' và import để dùng. Để tìm hiểu về nội dung source code và thực hành lại, các bạn cần truy nhập vào github của dự án.

Bài thực hành cần dùng package module1, có thể install từ github của dự án:

https://github.com/taile2007/Biosignal-analysis/tree/master/Data%20wrangling-01-Importation

Các bạn đặt module1 trong cùng thư mục hiện hành là có thể dùng được.

# Thông tin về phiên thực hành

In [1]:
import os
import sys
import re
import numpy as np
import pandas as pd
from module1.raw_data_parser import *

In [2]:
from sinfo import sinfo
sinfo()

-----
module1     0.1
numpy       1.19.4
pandas      1.1.5
pytz        2020.1
sinfo       0.3.1
tqdm        4.47.0
-----
IPython             7.16.1
jupyter_client      6.1.6
jupyter_core        4.6.3
jupyterlab          2.1.5
notebook            6.0.3
-----
Python 3.8.3 (default, Jul  2 2020, 17:30:36) [MSC v.1916 64 bit (AMD64)]
Windows-10-10.0.18362-SP0
16 logical CPU cores, Intel64 Family 6 Model 165 Stepping 2, GenuineIntel
-----
Session information updated at 2020-12-30 16:45


# Đặt vấn đề

Trong những thí nghiệm trên tín hiệu sinh học, dù là bài toán Machine learning, suy diễn thống kê hay đơn giản hơn nữa là mô tả/thăm dò, khó khăn hiển nhiên đó là ta phải làm việc trong một không gian chọn mẫu rất rộng. Như thí dụ này, dữ liệu thô là hàng chục Mb dữ liệu, kết quả của việc ghi các kênh tín hiệu trong gần 450 phút. Do đó, công đoạn đầu tiên chúng ta phải làm, đó là giới hạn, thu hẹp không gian này, và chuyên biệt hóa việc lấy mẫu. 

Những thủ thuật sau đây có thể được áp dụng:

1) Tái chọn mẫu ở một tần số lấy mẫu thấp hơn so với dữ liệu thô, vừa đủ để vẫn giữ được thông tin nhưng thu nhỏ kích thước của chuỗi tín hiệu (Under sampling)

2) Chia nhỏ chuỗi dữ liệu gốc thành những đoạn ngắn hơn, hay cửa sổ quan sát mỗi phân đoạn sẽ được xét như 1 đơn vị quan sát/chọn mẫu (Piece-wise sampling)

3) Chuyên biệt hóa: Tập trung khảo sát một số phân đoạn tương ứng với hiện tượng hay biến cố sinh lý bệnh mục tiêu. Để đạt được điều này, ta cần thực hiện công đoạn Dán nhãn (labeling).

Ta lập lại quy trình như bài 1, khai báo đường dẫn PACK_FOLDER, rồi dùng class Paths_Info_Summary và method create_signal_summary() để tạo ra bảng tóm tắt thông tin về gói dữ liệu thô:

In [3]:
PACK_FOLDER = 'D:\Project\Packs'

pack = Paths_Info_Summary(PACK_FOLDER)
pinfo = pack.create_signal_summary()

pinfo.head()

,full_paths,pack,filename,signal_type,Size (Mb)
0,D:\Project\Packs\Respiratory signal Example\Ra...,Respiratory signal Example,Abd RIP.txt,abd rip,3.304411
1,D:\Project\Packs\Respiratory signal Example\Ra...,Respiratory signal Example,Avg AP.txt,avg ap,4.410931
2,D:\Project\Packs\Respiratory signal Example\Ra...,Respiratory signal Example,Dia AP.txt,dia ap,4.401841
3,D:\Project\Packs\Respiratory signal Example\Ra...,Respiratory signal Example,ECG.txt,ecg,67.016486
4,D:\Project\Packs\Respiratory signal Example\Ra...,Respiratory signal Example,EMG1.txt,emg1,48.448591


# Chọn mẫu và dán nhãn bằng class DataSampler


## Mục tiêu giả định

Nhi đặt ra một tình huống giả định: ta cần thực hiện 1 thí nghiệm Machine learning với mục tiêu: phân tích tự động 3 kênh tín hiệu hô hấp, nhằm phân loại 3 trạng thái: Ngưng thở (Apnea), Giảm thở (Hypopnea), và Bình thường trong khi ngủ.

## Thiết kế quy trình chọn mẫu

Nhi thiết kế thí nghiệm như sau:

1) Ta sẽ giảm tần số lấy mẫu từ 32Hz xuống còn 10 Hz (kinh nghiệm cho thấy là vừa đủ để giữ lại các đặc tính của tín hiệu).

2) Ta đặt ra đơn vị quan sát (instance, observation unit) cho mô hình Machine learning là 1 cửa sổ trượt với kích thước = 10 giây. Ta sẽ di chuyển cửa sổ này dọc theo chiều dài của chuỗi tín hiệu, với tỉ lệ chồng lắp giữa 2 cửa sổ liên tiếp là 50% (chọn mẫu tăng cường) và dán nhãn biến cố hô hấp cho mỗi cửa sổ.

3) Ta lọc bỏ tất cả những phân đoạn trong trạng thái tỉnh thức (giả định là có 1 mô hình khác sẽ phân biệt Thức/Ngủ). 

## Mô tả quy trình

Sau đó, Nhi viết code để tạo ra 1 class Data_Sampler trong module raw_data_parser của package module1 để thi hành quy trình 10 bước:

**Bước 1**: Tải dữ liệu SpO2, ở samp_freq 10 Hz (vì trên lâm sàng, biến cố ngưng, giảm thở ngoài giảm biên độ tín hiệu lưu lượng thông khí, phải thỏa tiêu chí có kèm theo giảm bão hòa oxy máu).

**Bước 2**: Tải dữ liệu hypno, ở samp_freq tùy chọn = 10 Hz (để lọc bỏ những phân đoạn tỉnh thức)

**Bước 3**: Tải dữ liệu biến cố hô hấp (respi_evt), ở samp_freq tùy chọn (timestamp của mỗi biến cố sẽ được dùng để dán nhãn cho các cửa sổ)

**Bước 4**: Tải dữ liệu vi thức (ma), ở samp_freq tùy chọn (ở kích thước 10 Hz, biến cố vi thức có thể lọt vào trong cửa sổ quan sát, và vi thức là  biến cố độc lập với ngưng giảm thở, nên ta cho vào như 1 class thứ 4 của mô hình.

**Bước 5**: Kiểm tra, loại tất cả những biến cố hô hấp xảy ra trong khi thức (hypno label = 0)

**Bước 6**: Tải đồng loạt 4 kênh tín hiệu thô, và ghép lại thành dataframe signal_pack

**Bước 7**: Thực hiện chọn mẫu với 1 cửa sổ trượt kích thước = 100, có hoặc không có overlap

**Bước 8**: Dán nhãn vi thức, biến cố hô hấp và hypno cho mỗi cửa sổ quan sát

**Bước 9**: Tạo 1 danh sách biến cố hô hấp rút gọn, chỉ gồm những cửa sổ trong khi ngủ (hypno khác 0)

**Bước 10**: Tạo attribute patient_db, à 1 dictionary chứa toàn bộ nguyên liệu cần thiết cho thí nghiệm, và lưu lại như 1 attribute của class, bao gồm: 

        'ID' : tên bệnh nhân (data pack),
        'SpO2': chuỗi dữ liệu SpO2,
        'Raw_signal', : dataframe chứa 4 kênh tín hiệu: Flow_th, Abd_RIP, Tx_RIP, và OESP
        'Resp_evt': Thông tin về biến cố hô hấp,
        'Hypno': thông tin về trạng thái giấc ngủ,
        'MA': thông tin về biến cố vi thức,
        'Sampling': index của các cửa sổ chọn mẫu, kèm theo label của Resp và Hypno
        
Class DataSampler này còn cho phép sao lưu lại attribute patient_db này dưới định dạng pickle, để có thể tải và dùng trực tiếp mà không cần lặp lại 10 bước nêu trên.

Nhi trình diễn cách sử dụng class này như sau:

In [4]:
db = Data_Sampler(samp_freq = 10,
                 raw_data_folder = PACK_FOLDER,
                 df_paths = pinfo,
                 patient_id = 'Respiratory signal Example',
                  size = 100,
                 overlap = 0.5)

Tải thành công kênh SpO2
Tải thành công chuỗi Hypnogram
Tải thành công chuỗi biến cố hô hấp
Tải thành công chuỗi biến cố vi thức giấc
Đồng bộ hóa thành công tất cả kết quả PSG scoring
Khởi động tác vụ song song, tải 4 kênh tín hiệu


Tải và tái chọn mẫu thành công kênh tx rip
Tải và tái chọn mẫu thành công kênh flow th
Tải và tái chọn mẫu thành công kênh abd rip
Tải và tái chọn mẫu thành công kênh oesp

Đã tải xong 4 kênh tín hiệu
Hoàn tất tải dữ liệu thô
Áp dụng lấy mẫu tăng cường, kích thước = 100, tỉ lệ overlap = 0.5
Hoàn tất công đoạn lấy mẫu
Dán nhãn xong cho Biến cố vi thức
Dán nhãn xong cho Biến cố hô hấp
Dán nhãn xong cho Hypnogram


Khi gọi object db, ta biết tên bệnh nhân:

In [5]:
db

Đây là data base cho pack dữ liệu Respiratory signal Example

Kết quả của quy trình chọn mẫu, dán nhãn được lưu trong attribute patient_db:

In [6]:
db.patient_db.keys()

dict_keys(['ID', 'SpO2', 'Raw_signal', 'Resp_evt', 'Hypno', 'MA', 'Sampling'])

## Cấu trúc dữ liệu sau chọn mẫu

### Tên bệnh nhân

ID của bệnh nhân được lưu trong key 'ID', hoặc attribute db.patient_id

In [7]:
db.patient_db['ID']

'Respiratory signal Example'

In [8]:
db.patient_id

'Respiratory signal Example'

### Tín hiệu SpO2

Chuỗi định lượng liên tục có datetime index, định dạng pd.Series

In [9]:
db.patient_db['SpO2']

2017-09-30 22:52:18+02:00           97.0
2017-09-30 22:52:18.100000+02:00    97.0
2017-09-30 22:52:18.200000+02:00    97.0
2017-09-30 22:52:18.300000+02:00    97.0
2017-09-30 22:52:18.400000+02:00    97.0
                                    ... 
2017-10-01 06:19:23.500000+02:00    96.0
2017-10-01 06:19:23.600000+02:00    96.0
2017-10-01 06:19:23.700000+02:00    96.0
2017-10-01 06:19:23.800000+02:00    96.0
2017-10-01 06:19:23.900000+02:00    96.0
Freq: 100L, Name: spo2, Length: 268260, dtype: float64

### Dataframe 4 kênh tín hiệu hô hấp

Đây là dạng điển hình của dữ liệu chuỗi cho bài toán phân tích tín hiệu đa kênh, tất cả đều đồng bộ ở tần số 10 Hz và chung timestamps (SpO2 cũng vậy)

In [10]:
db.patient_db['Raw_signal']

,tx rip,flow th,abd rip,oesp
2017-09-30 22:52:18+02:00,-351.0,133.0,-22.0,-99.3
2017-09-30 22:52:18.100000+02:00,-670.0,48.0,-20.0,-99.4
2017-09-30 22:52:18.200000+02:00,-813.0,-22.0,-16.0,-99.4
2017-09-30 22:52:18.300000+02:00,-908.0,-91.0,-12.0,-100.4
2017-09-30 22:52:18.400000+02:00,-946.0,-160.0,-8.0,-99.8
...,...,...,...,...
2017-10-01 06:19:23.500000+02:00,-111.0,135.0,260.0,-100.3
2017-10-01 06:19:23.600000+02:00,-94.0,55.0,148.0,-99.6
2017-10-01 06:19:23.700000+02:00,-74.0,-8.0,64.0,-99.4
2017-10-01 06:19:23.800000+02:00,-50.0,-65.0,-12.0,-99.8


### Thông tin về các biến cố mục tiêu

3 keys 'Resp_evt', 'Hypno', 'MA' tương ứng với thông tin về biến cố hô hấp, trạng tái giấc ngủ, và vi thức, mỗi nhóm đều chứa một dictionary bên trong, với 4 keys là 'Evt_list', 'Raw', 'Label', 'Code'

In [11]:
db.patient_db['Resp_evt'].keys()

dict_keys(['Evt_list', 'Raw', 'Label', 'Code'])

'Evt_list' chứa danh sách biến cố, 1 dataframe có cấu trúc mỗi hàng là 1 biến cố, và 4 cột:'evt_start, 'evt_stop', 'evt_info'(thời gian mỗi biến cố) và 'evt_value' (nhãn kết quả)

In [13]:
db.patient_db['Resp_evt']['Evt_list']

,evt_start,evt_stop,evt_info,evt_value
0,2017-09-30 23:20:53.900000+02:00,2017-09-30 23:21:04.700000+02:00,10.9,4.0
1,2017-09-30 23:22:36.400000+02:00,2017-09-30 23:23:03.800000+02:00,27.5,3.0
2,2017-09-30 23:23:20.600000+02:00,2017-09-30 23:23:29.900000+02:00,9.4,4.0
3,2017-09-30 23:25:38.400000+02:00,2017-09-30 23:26:07.400000+02:00,29.1,2.0
4,2017-09-30 23:26:12.100000+02:00,2017-09-30 23:26:56.700000+02:00,44.7,2.0
...,...,...,...,...
81,2017-10-01 05:43:48.900000+02:00,2017-10-01 05:46:38+02:00,169.2,1.0
82,2017-10-01 05:46:38.400000+02:00,2017-10-01 05:46:55+02:00,16.7,3.0
83,2017-10-01 05:47:17.300000+02:00,2017-10-01 06:06:20.900000+02:00,1143.7,2.0
84,2017-10-01 06:06:28.400000+02:00,2017-10-01 06:06:40.300000+02:00,12.0,4.0


'Raw' chứa chuỗi định tính liên tục, được đồng bộ hóa với tín hiệu thô. Công dụng chính của định dạng này, dùng để vẽ biểu đồ, hoặc dán nhãn cho cửa sổ trượt sử dụng hàm Mode (yếu vị), tuy nhiên cách dán nhãn này không hay, bên dưới Nhi sẽ giải thích về algorithm dán nhãn dựa vào tỉ lệ trùng khớp khi đối chiếu timestamps của cửa sổ và của biến cố.

In [14]:
db.patient_db['Resp_evt']['Raw']

,value
2017-09-30 23:00:00+02:00,0
2017-09-30 23:00:00.100000+02:00,0
2017-09-30 23:00:00.200000+02:00,0
2017-09-30 23:00:00.300000+02:00,0
2017-09-30 23:00:00.400000+02:00,0
...,...
2017-10-01 06:15:30.400000+02:00,2
2017-10-01 06:15:30.500000+02:00,2
2017-10-01 06:15:30.600000+02:00,2
2017-10-01 06:15:30.700000+02:00,2


'Label' chứa kết quả của algorithm dán nhãn, nó có dạng 1 dataframe, với mỗi hàng là 1 cửa sổ (có 5364 cửa sổ đã được tạo ra), 

cột Resp_score chứa tỉ lệ trùng khớp (0.0 có nghĩa là cửa sổ hoàn toàn nằm bên ngoài biến cố, 1.0 có nghĩa là cửa sổ hoàn toàn rơi vào bên trong biến cố). Có thể dùng 1 ngưỡng như 0.7 hay 0.8 để lọc bỏ những cửa sổ không đặc thù, hay giai đoạn chuyển tiếp. 

Cột Resp_lab chứa giá trị của nhãn (label), ý nghĩa của các con số này được quy định trong db.patient_db['Resp_evt']['Code']

In [15]:
db.patient_db['Resp_evt']['Label']

,Resp_score,Resp_lab
0,0.0,0.0
1,0.0,0.0
2,0.0,0.0
3,0.0,0.0
4,0.0,0.0
...,...,...
5359,0.0,0.0
5360,0.0,0.0
5361,0.0,0.0
5362,0.0,0.0


In [16]:
db.patient_db['Resp_evt']['Code']

{'FL': 1, 'RERA': 2, 'OH': 3, 'CH': 4, 'OA': 5, 'CA': 6, 'MA': 8}

### Thông tin về indexing và labelling

key ['Sampling']['Label'] chứa 1 dataframe, với nội dung là tóm tắt thông tin indexing (cột start/end) của tất cả cửa sổ trong trạng thái ngủ (4867 cửa sổ), tỉ lệ trùng khớp (score) và giá trị label (lab) cho 2 loại biến cố: Hypno và Resp:

start/end tương ứng với thứ tự chính xác (đếm từ 0) trên chuỗi dữ liệu thô, từ đó có thể trích xuất đúng phân đoạn tín hiệu dài 10 s hay 100 giá trị cho mỗi cửa sổ.

In [17]:
db.patient_db['Sampling']['Label']

,start,end,Hypno_score,Hypno_lab,Resp_score,Resp_lab
0,16550,16650,30.0,1.0,0.0,0.0
1,16600,16700,80.0,1.0,0.0,0.0
2,16650,16750,100.0,1.0,0.0,0.0
3,16700,16800,100.0,1.0,0.0,0.0
4,16750,16850,100.0,1.0,0.0,0.0
...,...,...,...,...,...,...
4862,265650,265750,100.0,2.0,100.0,2.0
4863,265700,265800,100.0,2.0,100.0,2.0
4864,265750,265850,100.0,2.0,100.0,2.0
4865,265800,265900,100.0,2.0,100.0,2.0


key ['Sampling']['Windows'] chứa 1 dataframe với indexing start/end gốc cho toàn bộ 5364 cửa sổ chọn mẫu (chưa rút gọn). 

In [19]:
db.patient_db['Sampling']['Windows']

,start,end
0,0,100
1,50,150
2,100,200
3,150,250
4,200,300
...,...,...
5359,267950,268050
5360,268000,268100
5361,268050,268150
5362,268100,268200


## Algorithm dán nhãn

Để dán nhãn cho từng cửa sổ, dựa vào timestamp của chúng, Nhi áp dụng algorithm như sau:

Trước hết, tạo ra danh sách indexing cho toàn bộ cửa sổ theo trình tự thời gian (chứa trong key patient_db['Sampling']['Windows']),

Tại mỗi hàng, dùng giá trị start/end để lấy datetime index từ dataframe dữ liệu thô, và trích ra phân đoạn tương ứng.

Đối chiếu datetime index của phân đoạn này với timestamp evt_start/evt_end của 1 biến cố Đầu tiên trong danh sách biến cố;

Nếu cửa sổ hiện tại hoàn toàn nằm bên ngoài, và trước khi biến cố đầu tiên xảy ra, ta biết ngay label của nó = 0, và overlap score = 0; ta tiếp tục vòng lặp để đối chiếu cửa sổ tiếp theo với cùng biến cố.

Nếu cửa sổ hiện tại trùng khớp với 1 tỉ lệ nào đó > 0 với biến cố đầu tiên trong danh sách, ta dán cho nó label của biến cố đó, và tiếp tục kiểm tra cửa số tiếp theo,

Nếu cửa sổ hiện tại hoàn toàn vượt qua khỏi biến cố đầu tiên, tỉ lệ trùng khớp cũng xem như = 0, lúc này ta LOẠI BỎ biến cố đầu tiên khỏi danh sách, như vậy danh sách biến cố ngày càng ngắn lại, biến cố tiếp theo sẽ vượt lên hàng đầu.

Quy trình này được lặp lại cho đến khi toàn bộ danh sách biến cố đã bị xóa hết, nó trở nên rỗng. Lúc này mọi cửa sổ còn lại sẽ nhận label = 0, và overlap_score = 0.

# Lưu dữ liệu thành pickle file

method save_to_pickle() sẽ tạo ra 1 folder có tên là D_Respiratory trong thư mục hiện hành, bên trong có file DB_Respiratory.pickle có kích thước khoảng 31 Mb.

pickle là giải pháp khá hay để lưu trữ dữ liệu cho thí nghiệm này, và như ta thấy, bên trong file này sẽ chứa toàn bộ thông tin cần thiết cho những bước tiếp theo.

In [20]:
db.save_to_pickle()

Lưu thành công gói dữ liệu cho bệnh nhân Respiratory


# Tổng kết

Bài thực hành khép lại ở đây, trong bài này các bạn đã biết cách thiết kế và thực hiện 1 quy trình lấy mẫu và dán nhãn cho 1 bài toán Machine learning multiclass trên dữ liệu chuỗi tín hiệu đa kênh. Dù chỉ có tính chất giả định, quy trình làm việc này có thể được tùy biến dễ dàng cho những bài toán tương tự, hoặc một thí nghiệm mang tính chất mô tả, suy diễn thống kê.

Lưu ý: ngoài hình thức lấy mẫu dạng cửa sổ trượt với độ dài cố định, có kèm hay không kèm overlap, còn có nhiều cách lấy mẫu khác, thí dụ lấy mẫu từng cặp 2-3 cửa sổ liên tiếp (đơn vị quan sát đích +/- 1 khoảng trước hoặc sau). Trong các quy trình deep learning, có thể xây dựng data generator để sinh batch dữ liệu ở thời gian thực ngay trong khi huấn luyện; hoặc chuyển dữ liệu thô thành những tấm ảnh (spectrogram, scaleogram...). Những cách làm này này quá phức tạp nên không được chọn áp dụng trong bài.

Hẹn gặp các bạn trong bài tiếp theo.